In [ ]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
import itertools
import warnings

In [ ]:
def mmn_queueing_jsq(n,arrival_rate,service_rate,simulation_time=60,simuseed=math.nan):
    '''
    Simulating the mmn queueing system using the method of JSQ 
    (namely join the shortest queue method), the user needs to provide
    the number of servers (n),arrival rate (lambda) and service rate (mu),
    in case of n >= 2, if mu is provided as a scaler, then all servers
    share the same mu, if mu is provided as a vector, then all servers may have different mu,
    the simulation time is by default 60 units, and for reproducibility purposes, the user
    can define simulation seed, by default the seed value is not set.
    
    There are 5 inputs in this function,
    n:                the number of servers, a positive finite integer
    arrival_rate:     exponential rate of arrival
    service_rate:     exponential rate of service time
    simulation_time:  the simulation time, a positive finite number
    simuseed:         the simulation seed number
    
    
    There are 8 outputs in this function,
    event_calendar:   the event calendar for the whole simulation, containing all important
                      information needed (current time, next customer arrival time, finish time,
                      server occupancy variable, server next available time, server queue length,
                      server total, grand queue length and grand total)
    utiliz:           the utilization for each server
    ans:              the average number in the system for each server and grand average
    aql:              the average queue length for each server and grand average
    awt:              the average waiting time for each customer and grand average
    act:              the average cycle time for each customer and grand average
    no_cust_arr_comp: the table illustrating the number of arrived customers and the
                      the number of completed services for each server and grand total
    other_stat:       return other calculated statistics, for plotting purposes
    
    Code writer: Yaozu Li
    uoe S.No: 2298215
    
    '''
    
    if math.isnan(simuseed) == True:
        pass
    else:
        random.seed(simuseed)

    if not type(n) is int:
        raise TypeError("n needs to be integer")
    if n < 0:
        raise Exception("n needs to be positive")
    
    lamb =  arrival_rate
    
    if isinstance(service_rate, list):
        if len(service_rate) == n:
            mu = service_rate
        elif (n > 1 and len(service_rate) == 1):
            mu = n*service_rate
        else:
            raise Exception("Unmatched cardinality for service rate and n")
    elif isinstance(service_rate, int):
        if n == 1:
            mu = [service_rate]
        elif n > 1:
            mu = n*[service_rate]
    else:
        raise Exception("Please use int or list type for service rate")
    
    
    # Simulating the event calendar
    table_col = np.arange(n)*4+3
    avail_col = np.arange(n)*4+4
    queue_col = np.arange(n)*4+5
    total_col = np.arange(n)*4+6
    compare_col = np.insert(avail_col,0,[1,2])
    
    event_calendar = np.zeros([2,3+4*n])
    event_calendar[-1,1] = random.expovariate(lamb)
    event_calendar[-1,2] = simulation_time
    event_calendar[-1,avail_col] = math.nan
    
    no_customers = np.zeros(n+1)  # keep track of the number customers
    no_completed_service = np.zeros(n+1)   # keep track of the number of completed services
    
    while event_calendar[-1,0] < simulation_time:
        event_calendar = np.vstack([event_calendar,event_calendar[-1,:]])
        event_calendar[-1,0] = min(event_calendar[-2,compare_col])
        
        # If the next event is an arrival
        if event_calendar[-1,0] == event_calendar[-2,1]:
            event_calendar[-1,1] = event_calendar[-1,0]+random.expovariate(lamb)
            server_assignment = random.choice(np.where(event_calendar[-1,total_col] == \
                                   min(event_calendar[-1,total_col]))[0])
            no_customers[server_assignment] += 1
            
            # If the server is not occupied
            if event_calendar[-1,table_col[server_assignment]] == 0:
                event_calendar[-1,table_col[server_assignment]] += 1
                event_calendar[-1,avail_col[server_assignment]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[server_assignment])
                event_calendar[-1,total_col[server_assignment]] += 1
            
            # Else if the server is occupied
            elif event_calendar[-1,table_col[server_assignment]] == 1:
                event_calendar[-1,queue_col[server_assignment]] += 1
                event_calendar[-1,total_col[server_assignment]] += 1
            
        # Else if the next event is the simulation termination
        elif event_calendar[-1,0] == simulation_time:
            pass
        
        # Else if the next event is a service completion
        else:
            server_completion = np.where(event_calendar[-2,avail_col] == event_calendar[-1,0])[0][0]
            no_completed_service[server_completion] += 1
            
            # If there is no queue behind
            if event_calendar[-1,queue_col[server_completion]] == 0:
                event_calendar[-1,table_col[server_completion]] -= 1
                event_calendar[-1,avail_col[server_completion]] = math.nan
                event_calendar[-1,total_col[server_completion]] -= 1
            
            # Else if there is a queue behind
            elif event_calendar[-1,queue_col[server_completion]] > 0:
                event_calendar[-1,avail_col[server_completion]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[server_completion])
                event_calendar[-1,queue_col[server_completion]] -= 1
                event_calendar[-1,total_col[server_completion]] -= 1
    
    no_customers[-1] = sum(no_customers[:-1])
    no_completed_service[-1] = sum(no_completed_service[:-1])
    event_calendar = pd.DataFrame(event_calendar).tail(-1)
    event_calendar.columns = ['Time','Next Customer','Finish Time'] + \
                    list(itertools.chain.from_iterable([[f"Server{i}", \
                    f"Server{i} Available Time",f"Server{i} Queue",f"Server{i} Total"] for i in range(1,n+1)]))
    event_calendar["Grand Queue"] = event_calendar.iloc[:,queue_col].sum(axis=1)
    event_calendar["Grand Total"] = event_calendar.iloc[:,total_col].sum(axis=1)
    
    # Concluding performance measures
    # Time dynamic
    time1 = event_calendar.iloc[:-1,0].to_numpy()
    time2 = event_calendar.iloc[1:,0].to_numpy()
    time3 = time2 - time1
    
    # Calculate the utilization for each server
    utiliz_server = event_calendar.iloc[:-1,table_col].to_numpy()
    utiliz_data = ((time3 @ utiliz_server)/simulation_time).round(3)
    utiliz = pd.DataFrame(utiliz_data).transpose()
    utiliz.columns = [f"Server{i} Utilization" for i in range(1,n+1)]
    
    # Calculating the average number in the systems for each server and grand total
    ans_server = event_calendar.iloc[:-1,np.append(total_col,event_calendar.shape[1]-1)].to_numpy()
    ans_data = ((time3 @ ans_server)/simulation_time).round(3)
    ans = pd.DataFrame(ans_data).transpose()
    ans.columns = [f"Server{i} Average number in the system" for i in range(1,n+1)]+["Grand Average number in the system"]
    
    # Calculating the average queue length
    aql_server = event_calendar.iloc[:-1,np.append(queue_col,event_calendar.shape[1]-2)].to_numpy()
    aql_data = ((time3 @ aql_server)/simulation_time).round(3)
    aql = pd.DataFrame(aql_data).transpose()
    aql.columns = [f"Server{i} Average queue length" for i in range(1,n+1)]+["Grand Average queue length"]
    
    # Average waiting time
    awt_data = ((time3 @ aql_server)/no_customers).round(3)
    awt = pd.DataFrame(awt_data).transpose()
    awt.columns = [f"Server{i} Average waiting time" for i in range(1,n+1)]+["Grand Average waiting time"]
    
    # Average cycle time
    act_data = ((time3 @ ans_server)/no_customers).round(3)
    act = pd.DataFrame(act_data).transpose()
    act.columns = [f"Server{i} Average cycle time" for i in range(1,n+1)]+["Grand Average cycle time"]
    
    # Show the number of customers arrived and completed
    no_cust_arr_comp = pd.DataFrame(np.vstack([no_customers,no_completed_service]))
    no_cust_arr_comp.columns = [f"Server{i}" for i in range(1,n+1)]+["Grand total"]
    no_cust_arr_comp.index = ["Arrived customers", "Completed services"]
    
    # Find the maximum queue length
    mql = event_calendar.iloc[:,queue_col].max()
    
    # For other statistics
    other_stat = {
        "n": n,
        "lamb": lamb,
        "mu": mu,
        "simulation_time": simulation_time,
        "table_col": table_col,
        "avail_col": avail_col,
        "queue_col": queue_col,
        "total_col": total_col,
        "compare_col": compare_col,
        "time3": time3,
        "mql": mql
    }
    
    # Return the event calendar and all performance measures table
    return event_calendar, utiliz, ans, aql, awt, act, no_cust_arr_comp, other_stat


In [ ]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[0]

In [ ]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[1]

In [ ]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[2]

In [ ]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[3]

In [ ]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[4]

In [ ]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[5]

In [ ]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[6]

In [ ]:
def queue_length_time_graph_jsq(event_calendar,other_stat,gtype="queue",sq_no=math.nan):
    '''
    A function to plot the queue length time graph, provided with an event calendar
    and other_stat which are produced by the mmn_queueing_jsq function
    
    There are x inputs in this function,
    event_calendar:   the event calendar that was produced by the mmn_queueing_jsq function
    other_stat:       the dictionary other_stat that was produced by the mmn_queueing_jsq function
    gtype:            choose the type of graph you need, there are two options, the "queue" type
                      is by default, and it generates the queue length versus time for
                      each server. The "server" type generates the number of servers versus time
                      for each queue length.
    sq_no:            by default, only the server1 or the queue length of 0 will be plotted,
                      depending on the gtype. Unless specify it here, only scaler or list type input
                      are accepted.
                      
    The outputs are the corresponding requested graphs.
    
    code writer: Yaozu Li
    uoe S.No: 2298215
    
    '''
    
    # Preprocessing
    if isinstance(sq_no, int):
        if gtype == "queue":
            if sq_no > other_stat["n"] or sq_no < 1:
                raise Exception("request out of range")
            else:
                sq_no = [sq_no]
        elif gtype == "server":
            if sq_no < 0:
                raise Exception("only positive queue length exists")
            else:
                sq_no = [sq_no]
        else:
            raise TypeError("Please use queue or server for gtype")
    elif isinstance(sq_no, list) or isinstance(sq_no, np.ndarray):
        if gtype == "queue":
            if max(sq_no) > other_stat["n"] or min(sq_no) < 1:
                raise Exception("request out of range")
            else:
                pass
        elif gtype == "server":
            if min(sq_no) < 0:
                raise Exception("only positive queue length exists")
            else:
                pass
        else:
            raise TypeError("Please use queue or server for gtype")
    elif math.isnan(sq_no) == True:
        if gtype == "queue":
            sq_no = [1]
        elif gtype == "server":
            sq_no = [0]
        else:
            raise TypeError("Please use queue or server for gtype")
    else:
        raise Exception("Please use int, list or np.array type for sq_no")
    
    # If gtype = "queue"
    if gtype == "queue":
        for i in sq_no:
            plt.figure(i)
            plt.plot(event_calendar["Time"],event_calendar[f"Server{i} Queue"])
            plt.title(f"Server{i}")
            plt.xlabel("Time")
            plt.ylabel("Queue length")
    else:
        for i in sq_no:
            plt.figure(i)
            plt.plot(event_calendar["Time"],(event_calendar.iloc[:,other_stat["queue_col"]] == i).sum(axis=1).to_numpy())
            plt.title(f"Queue length of {i}")
            plt.xlabel("Time")
            plt.ylabel("The number of servers")
    

In [ ]:
sybb = mmn_queueing_jsq(5,30,[4,2,5,3,3],simulation_time=10,simuseed=8)

In [ ]:
queue_length_time_graph_jsq(sybb[0],sybb[7],gtype="queue",sq_no=np.arange(1,6))